# 해야 할 일

- 논문 검색 (google scholar, ieee xplore, dbpia 등)<br/>
  (관련 키워드 : deep learning, detection, danger ....)
- 초록 작성 : 찾아본 논문들 참고해서 한글 혹은 영어로 작성
- 이미지 수집 및 전처리 (32,32,3 이미지 리사이징, 폴더별로 정리)

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(32, 32),
        batch_size=3,
        class_mode='binary') 

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(32, 32),     
        batch_size=3,
        class_mode='binary')



Found 380 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [3]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32,32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax')) 


# 3. 모델 학습과정 설정하기
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=10, 
        epochs=500,  
        validation_data=test_generator,
        validation_steps=5)

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Epoch 1/500
10/10 [==============================] - 2s 152ms/step - loss: 11.2444 - accuracy: 0.2667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 2/500
10/10 [==============================] - 0s 17ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 3/500
10/10 [==============================] - 0s 17ms/step - loss: 13.7999 - accuracy: 0.1000 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 4/500
10/10 [==============================] - 0s 15ms/step - loss: 8.6888 - accuracy: 0.4333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 5/500
10/10 [==============================] - 0s 17ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 6/500
10/10 [==============================] - 0s 15ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 7/500
10/10 [==============================] - 0s 18ms/step - loss: 11.7555 - accuracy: 0.2333 - val_loss:

Epoch 56/500
10/10 [==============================] - 0s 18ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 57/500
10/10 [==============================] - 0s 18ms/step - loss: 12.2666 - accuracy: 0.2000 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 58/500
10/10 [==============================] - 0s 14ms/step - loss: 11.7443 - accuracy: 0.2414 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 59/500
10/10 [==============================] - 0s 16ms/step - loss: 12.2666 - accuracy: 0.2000 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 60/500
10/10 [==============================] - 0s 15ms/step - loss: 11.7555 - accuracy: 0.2333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 61/500
10/10 [==============================] - 0s 15ms/step - loss: 14.3110 - accuracy: 0.0667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 62/500
10/10 [==============================] - 0s 17ms/step - loss: 9.7111 - accuracy: 0.3667 - val

Epoch 111/500
10/10 [==============================] - 0s 19ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 112/500
10/10 [==============================] - 0s 16ms/step - loss: 13.7999 - accuracy: 0.1000 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 113/500
10/10 [==============================] - 0s 18ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 114/500
10/10 [==============================] - 0s 18ms/step - loss: 11.7555 - accuracy: 0.2333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 115/500
10/10 [==============================] - 0s 21ms/step - loss: 12.2666 - accuracy: 0.2000 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 116/500
10/10 [==============================] - 0s 17ms/step - loss: 13.2888 - accuracy: 0.1333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 117/500
10/10 [==============================] - 0s 14ms/step - loss: 11.2444 - accuracy: 0.26

10/10 [==============================] - 0s 16ms/step - loss: 12.7777 - accuracy: 0.1667 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 166/500
10/10 [==============================] - 0s 17ms/step - loss: 13.2888 - accuracy: 0.1333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 167/500
10/10 [==============================] - 0s 15ms/step - loss: 12.2666 - accuracy: 0.2000 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 168/500
10/10 [==============================] - 0s 17ms/step - loss: 13.2888 - accuracy: 0.1333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 169/500
10/10 [==============================] - 0s 19ms/step - loss: 13.2888 - accuracy: 0.1333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 170/500
10/10 [==============================] - 0s 17ms/step - loss: 13.2888 - accuracy: 0.1333 - val_loss: 15.3332 - val_accuracy: 0.0000e+00
Epoch 171/500
10/10 [==============================] - 0s 18ms/step - loss: 11.2444 - accuracy: 0.2667 - val_loss:

Epoch 220/500


KeyboardInterrupt: 